<a href="https://colab.research.google.com/github/hrnrxb/Advanced-Sentiment-Classifier/blob/main/%5B%F0%9F%94%A5_Advanced_Sentiment_Classifier%5D_RoBERTa_%2B_BiLSTM_%2B_Attention_(Handles_Sarcasm_%26_Contrast!).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets gradio


In [1]:
import torch
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaModel
from datasets import load_dataset
import gradio as gr
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)


Using: cuda


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!pip install --upgrade datasets huggingface_hub fsspec


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.2
    Uninstalling huggingface-hub-0.33.2:
      Successfully uninstalled huggingface-hub-0.33.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_s

In [2]:
from datasets import load_dataset

dataset = load_dataset("stanfordnlp/imdb")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

dataset = load_dataset("imdb")  # می‌تونی بعداً با sarcasm یا SST-5 جایگزین کنی

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

encoded_dataset = dataset.map(tokenize, batched=True)
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
class RobertaBiLSTMAttention(nn.Module):
    def __init__(self, hidden_dim=128, num_labels=2):
        super().__init__()
        self.roberta = RobertaModel.from_pretrained("roberta-base")
        self.lstm = nn.LSTM(768, hidden_dim, batch_first=True, bidirectional=True)
        self.attn = nn.Linear(hidden_dim * 2, 1)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim * 2, num_labels)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            roberta_out = self.roberta(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        lstm_out, _ = self.lstm(roberta_out)
        weights = torch.softmax(self.attn(lstm_out), dim=1)
        context = torch.sum(weights * lstm_out, dim=1)
        output = self.fc(self.dropout(context))
        return output


In [6]:
from torch.utils.data import DataLoader

train_loader = DataLoader(encoded_dataset["train"].select(range(20000)), batch_size=16, shuffle=True)
test_loader = DataLoader(encoded_dataset["test"].select(range(2000)), batch_size=16)


In [8]:
from tqdm import tqdm

model = RobertaBiLSTMAttention().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

for epoch in range(10):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for batch in progress_bar:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        avg_loss = total_loss / (progress_bar.n if progress_bar.n else 1)
        progress_bar.set_postfix(loss=f"{avg_loss:.4f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 10: 100%|██████████| 1250/1250 [02:17<00:00,  9.10it/s, loss=0.2727]


In [15]:
model.eval()

samples = [
    # 1: sarcasm
    "Wow, what a masterpiece. I especially loved the part where nothing happened for two hours.",  # 🔴 Negative

    # 2: emotionally mixed
    "The movie was boring at times, but the ending completely blew my mind.",  # 🟢 Positive (نسبتاً مثبت)

    # 3: ta’arofy tone (fake praise)
    "It was... fine, I guess. Not bad. Not good. Just there.",  # 🔴 Negative (neutral leaning negative)

    # 4: visually good but poor content
    "Beautiful cinematography can’t save a script written by a potato.",  # 🔴 Negative

    # 5: praise with weird slang
    "Yo that movie was sick af! 🔥🔥",  # 🟢 Positive ("sick" = slang for amazing)

    # 6: backhanded compliment
    "I didn’t expect much, and yet it still managed to disappoint me.",  # 🔴 Negative

    # 7: full sarcasm
    "10/10 would recommend... if you enjoy falling asleep halfway through.",  # 🔴 Negative

    # 8: fake excitement
    "Absolutely incredible! I only checked my phone 12 times.",  # 🔴 Negative

    # 9: nostalgic + honest
    "Reminded me of my childhood, cheesy but heartwarming.",  # 🟢 Positive

    # 10: hype tone
    "Bro that film went HARD. Straight banger!",  # 🟢 Positive (slang-heavy positive)
]



for s in samples:
    tokens = tokenizer(s, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)
    with torch.no_grad():
        logits = model(tokens["input_ids"], tokens["attention_mask"])
        pred = torch.argmax(logits, dim=1).item()
        print(f"{s} ➤ {'🟢 Positive' if pred == 1 else '🔴 Negative'} ")


Wow, what a masterpiece. I especially loved the part where nothing happened for two hours. ➤ 🟢 Positive 
The movie was boring at times, but the ending completely blew my mind. ➤ 🔴 Negative 
It was... fine, I guess. Not bad. Not good. Just there. ➤ 🔴 Negative 
Beautiful cinematography can’t save a script written by a potato. ➤ 🔴 Negative 
Yo that movie was sick af! 🔥🔥 ➤ 🔴 Negative 
I didn’t expect much, and yet it still managed to disappoint me. ➤ 🔴 Negative 
10/10 would recommend... if you enjoy falling asleep halfway through. ➤ 🟢 Positive 
Absolutely incredible! I only checked my phone 12 times. ➤ 🟢 Positive 
Reminded me of my childhood, cheesy but heartwarming. ➤ 🟢 Positive 
Bro that film went HARD. Straight banger! ➤ 🟢 Positive 


In [11]:
def predict(text):
    model.eval()
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)
    with torch.no_grad():
        logits = model(tokens["input_ids"], tokens["attention_mask"])
        prob = torch.softmax(logits, dim=1)
        pred = torch.argmax(prob, dim=1).item()
        conf = prob[0][pred].item()
    label = "🟢 Positive" if pred == 1 else "🔴 Negative"
    return f"{label} ({conf*100:.1f}%)"

gr.Interface(fn=predict, inputs=gr.Textbox(label="Enter a review"), outputs="text").launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6d77e8526c5a0f6d82.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [17]:
import os

os.makedirs("model", exist_ok=True)  # اگر پوشه وجود نداشته باشه، می‌سازه

torch.save(model.state_dict(), "model/pytorch_model.bin")


In [19]:
torch.save(model.state_dict(), "model/pytorch_model.bin")


In [20]:
from google.colab import files
files.download("/content/model/pytorch_model.bin")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/model/pytorch_model.bin /content/drive/MyDrive/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
